In [1]:
print('hello')

hello


In [3]:

import os
from dotenv import load_dotenv

load_dotenv()
GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY")

In [6]:
# Function to load documents from a directory
def load_documents_from_directory(directory_path):
    print("==== Loading documents from directory ====")
    documents = []
    for filename in os.listdir(directory_path):
        if filename.endswith(".txt"):
            with open(
                os.path.join(directory_path, filename), "r", encoding="utf-8"
            ) as file:
                documents.append({"id": filename, "text": file.read()})
    return documents

# Function to load documents from a file
directory_path = './news_articles'
documents = load_documents_from_directory(directory_path)
print(f"Loaded {len(documents)} documents from {directory_path}")

==== Loading documents from directory ====
Loaded 21 documents from ./news_articles


In [7]:
#Function to Split Documents into chunks
def split_text(text, chunk_size=1000, chunk_overlap=20):
    chunks = []
    start = 0
    while start < len(text):
        end = start + chunk_size
        chunks.append(text[start:end])
        start = end - chunk_overlap
    return chunks

#splitting the documents into chunks
chunked_documents = []
for doc in documents:
    chunks = split_text(doc["text"])
    print("==== Splitting docs into chunks ====")
    for i, chunk in enumerate(chunks):
        chunked_documents.append({"id": f"{doc['id']}_chunk{i+1}", "text": chunk})

==== Splitting docs into chunks ====
==== Splitting docs into chunks ====
==== Splitting docs into chunks ====
==== Splitting docs into chunks ====
==== Splitting docs into chunks ====
==== Splitting docs into chunks ====
==== Splitting docs into chunks ====
==== Splitting docs into chunks ====
==== Splitting docs into chunks ====
==== Splitting docs into chunks ====
==== Splitting docs into chunks ====
==== Splitting docs into chunks ====
==== Splitting docs into chunks ====
==== Splitting docs into chunks ====
==== Splitting docs into chunks ====
==== Splitting docs into chunks ====
==== Splitting docs into chunks ====
==== Splitting docs into chunks ====
==== Splitting docs into chunks ====
==== Splitting docs into chunks ====
==== Splitting docs into chunks ====


In [8]:
len(chunked_documents)

184

In [15]:
from langchain_community.embeddings import HuggingFaceEmbeddings
from chromadb.utils.embedding_functions import EmbeddingFunction

# Wrapper to make LangChain HuggingFaceEmbeddings compatible with ChromaDB
class ChromaCompatibleHFEmbedding(EmbeddingFunction):
    def __init__(self, model_name="sentence-transformers/all-MiniLM-L6-v2"):
        self.model = HuggingFaceEmbeddings(model_name=model_name)

    def __call__(self, input):
        return self.model.embed_documents(input)

# Instantiate the wrapper
embedding_function = ChromaCompatibleHFEmbedding()


In [16]:
import chromadb

# Initialize the Chroma client with persistence
chroma_client = chromadb.PersistentClient(path="chroma_persistent_storage")
collection_name = "document_qa_collection"
collection = chroma_client.get_or_create_collection(
    name=collection_name, embedding_function=embedding_function
)

In [20]:
from tqdm import tqdm  # for progress bars

def generate_and_store_embeddings(documents, embedding_function, collection):
    print("==== Generating embeddings and storing in ChromaDB ====")

    ids = []
    texts = []
    embeddings = []

    for doc in documents:
        ids.append(doc["id"])
        texts.append(doc["text"])
        embedding = embedding_function([doc["text"]])[0]  # Proper call
        embeddings.append(embedding)

    collection.add(
        ids=ids,
        documents=texts,
        embeddings=embeddings
    )

    print(f"✅ Stored {len(texts)} chunks in ChromaDB.")



# Assume embeddings and collection are already initialized
#generate_and_store_embeddings(chunked_documents, embedding_function, collection)


In [21]:

# Assume embeddings and collection are already initialized
generate_and_store_embeddings(chunked_documents, embedding_function, collection)

==== Generating embeddings and storing in ChromaDB ====
✅ Stored 184 chunks in ChromaDB.


In [28]:

# Function to query documents
def query_documents(question, n_results=2):
    # query_embedding = get_openai_embedding(question)
    results = collection.query(query_texts=question, n_results=n_results)

    # Extract the relevant chunks
    relevant_chunks = [doc for sublist in results["documents"] for doc in sublist]
    print("==== Returning relevant chunks ====")
    return relevant_chunks
    # for idx, document in enumerate(results["documents"][0]):
    #     doc_id = results["ids"][0][idx]
    #     distance = results["distances"][0][idx]
    #     print(f"Found document chunk: {document} (ID: {doc_id}, Distance: {distance})")

In [22]:
#initializing the llm
from langchain_google_genai import ChatGoogleGenerativeAI

llm = ChatGoogleGenerativeAI(model="gemini-2.0-flash",
                            google_api_key=GOOGLE_API_KEY , 
                            temperature=0.7)


In [30]:
from langchain_google_genai import ChatGoogleGenerativeAI

llm = ChatGoogleGenerativeAI(model="gemini-1.5-flash", temperature=0.3)

def generate_response(question, relevant_chunks):
    context = "\n\n".join(relevant_chunks)
    prompt = (
        "You are an assistant for question-answering tasks. Use the following pieces of "
        "retrieved context to answer the question. If you don't know the answer, say that you "
        "don't know. Use three sentences maximum and keep the answer concise."
        "\n\nContext:\n" + context + "\n\nQuestion:\n" + question
    )

    response = llm.invoke(prompt)  # LangChain's way
    return response.content


In [31]:
# Example query
# query_documents("tell me about AI replacing TV writers strike.")
# Example query and response generation
question = "tell me about databricks"
relevant_chunks = query_documents(question)
answer = generate_response(question, relevant_chunks)

print(answer)

==== Returning relevant chunks ====
Databricks acquired Okera, a data governance platform focused on AI, to improve its Unity Catalog governance solution.  This acquisition addresses the challenges of managing rapidly expanding data assets in the age of LLMs.  Databricks also plans to integrate Okera's technology and expose additional APIs for its data governance partners.
